In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Naive Bayesian Fusion with Accelerated Approximation
----------------------------------------------------
This script implements a probability fusion algorithm based on 
a Naive Bayesian approach. It supports both:

  • Exact fusion (enumerates all possible combinations).
  • Accelerated fusion (sampling-based approximation).

The accelerated algorithm is useful for large input lists where 
exact enumeration is computationally expensive.

License
-------
This code is released under the MIT License (see below).
Feel free to use, modify, and distribute with attribution.

Author: Xiaoyu Liu
Repository: https://github.com/xiaoyuliu-space/Fast_naive_Bayesian_fusion
"""

import itertools
import math
import time
import numpy as np

# -----------------------------
# Core exact fusion
# -----------------------------
def _exact_fuse_probabilities(probs: np.ndarray) -> float:
    if probs.size == 0:
        return 0.0

    p1 = np.asarray(probs, dtype=float)
    if np.any((p1 < 0) | (p1 > 1)):
        raise ValueError("All probabilities must be within [0, 1].")

    p0 = 1.0 - p1
    n = p1.size
    qn = 1 if n >= 1 else 0

    overall = 0.0
    indices = list(range(n))

    for i in range(n + 1):
        for comb in itertools.combinations(indices, i):
            product = 1.0
            weight_adjust = 0
            in_comb = set(comb)

            for j in indices:
                if j in in_comb:
                    product *= p1[j]
                    if p1[j] >= 0.5:
                        weight_adjust = 1
                else:
                    product *= p0[j]

            frac = i / n
            probC1 = max(weight_adjust, frac)
            probC0 = max(math.ceil(frac), 0.5)
            probC = probC1 * qn + probC0 * (1 - qn)

            overall += product * probC

    return float(overall)


# -----------------------------
# Accelerated sampling helper
# -----------------------------
def _stratified_sample(probs: np.ndarray, sample_size: int, threshold: float, rng: np.random.Generator) -> np.ndarray:
    if probs.size <= sample_size:
        return probs.copy()

    upper = probs[probs >= (1 - threshold)]
    lower = probs[probs < threshold]

    num_upper = int(round(sample_size * (upper.size / probs.size))) if probs.size > 0 else 0
    num_upper = min(num_upper, upper.size)
    num_lower = sample_size - num_upper
    num_lower = min(num_lower, lower.size)

    chosen = []
    if upper.size > 0 and num_upper > 0:
        chosen.append(rng.choice(upper, size=num_upper, replace=False))
    if lower.size > 0 and num_lower > 0:
        chosen.append(rng.choice(lower, size=num_lower, replace=False))

    remaining = sample_size - sum(len(x) for x in chosen) if chosen else sample_size
    if remaining > 0:
        already = np.concatenate(chosen) if chosen else np.array([], dtype=float)
        counts = {}
        for v in already:
            counts[v] = counts.get(v, 0) + 1
        remaining_pool = []
        for v in probs:
            if counts.get(v, 0) > 0:
                counts[v] -= 1
            else:
                remaining_pool.append(v)
        remaining_pool = np.asarray(remaining_pool, dtype=float)
        pick_n = min(remaining, remaining_pool.size)
        if pick_n > 0:
            chosen.append(rng.choice(remaining_pool, size=pick_n, replace=False))

    return np.concatenate(chosen) if chosen else probs[:sample_size]


def _accelerated_fuse_probabilities(
    probs: np.ndarray,
    threshold: float = 0.5,
    sample_size: int = 5,
    tol_ratio: float = 0.01,
    max_iter: int = 1000,
    seed: int | None = None
):
    if probs.size == 0:
        return 0.0, [], []

    rng = np.random.default_rng(seed)
    estimates = []
    errors = []
    if probs.size <= sample_size:
        fused = _exact_fuse_probabilities(probs)
        return fused, [fused], [0.0]

    running_mean = None
    for it in range(max_iter):
        sample = _stratified_sample(probs, sample_size=sample_size, threshold=threshold, rng=rng)
        est = _exact_fuse_probabilities(sample)
        estimates.append(est)

        if running_mean is None:
            running_mean = est
        else:
            running_mean = running_mean + (est - running_mean) / len(estimates)

        err = abs(est - running_mean)
        errors.append(err)

        denom = abs(running_mean) if running_mean != 0 else 1.0
        if err <= tol_ratio * denom and it >= 5:
            break

    return estimates[-1], estimates, errors


# -----------------------------
# Public function
# -----------------------------
def fuse_probabilities(
    prob_list,
    *,
    accelerate: bool = True,
    threshold: float = 0.5,
    sample_size: int = 5,
    tol_ratio: float = 0.01,
    max_iter: int = 1000,
    seed: int | None = None
) -> float:
    probs = np.asarray(list(prob_list), dtype=float)
    if probs.ndim != 1:
        raise ValueError("Input must be a 1D list/array of probabilities.")
    if np.any((probs < 0) | (probs > 1)):
        raise ValueError("All probabilities must be within [0, 1].")

    if not accelerate or probs.size <= 12:
        return _exact_fuse_probabilities(probs)

    fused, _, _ = _accelerated_fuse_probabilities(
        probs,
        threshold=threshold,
        sample_size=sample_size,
        tol_ratio=tol_ratio,
        max_iter=max_iter,
        seed=seed
    )
    return fused


# -----------------------------
# Example usage
# -----------------------------
if __name__ == "__main__":
    # easily generate random input list
    input_list = np.random.rand(25).tolist()

    t0 = time.perf_counter()
    fused_exact = fuse_probabilities(input_list, accelerate=False)
    t1 = time.perf_counter()

    fused_fast = fuse_probabilities(input_list, accelerate=True, seed=42)
    t2 = time.perf_counter()

    print("Exact fused:", fused_exact)
    print("Accelerated fused:", fused_fast)
    print(f"Exact time      : {t1 - t0:.6f} s")
    print(f"Accelerated time: {t2 - t1:.6f} s")
    # last line: time saved
    print(f"Time saved by accelerated algorithm: {(t1 - t0) - (t2 - t1):.6f} s")


Exact fused: 0.9999998894326695
Accelerated fused: 0.9395948324569728
Exact time      : 87.626295 s
Accelerated time: 0.001055 s
Time saved by accelerated algorithm: 87.625239 s
